# Agent Supervisor Architecture Application
Applying a supervisor approach to ideas of social media sentiment analysis

In [1]:
import os
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## Gathering API keys

OpenAI and LangChain API key set in environment variable, `langchain_tracing_v2` allows langsmith agent tracing, `langchain_project` determines organization in langsmith

In [5]:
langchain_tracing_v2 = os.environ["LANGCHAIN_TRACING_V2"]
langchain_api_key = os.environ["LANGCHAIN_API_KEY"]
langchain_project = os.environ["LANGCHAIN_PROJECT"]
# os.environ["OPENAI_API_KEY"] = "apikey"

In [6]:
llm = Ollama(model="llama3")

In [12]:
llm.invoke("how can langsmith help with testing?")

"LangSmith is a powerful language model that can be used to assist with testing in various ways. Here are some examples:\n\n1. **Automated Testing**: LangSmith can generate test cases and input data based on the code under test, allowing for more comprehensive and efficient testing.\n2. **Code Review**: LangSmith can analyze code and provide feedback on coding best practices, potential issues, and suggestions for improvement, helping to identify defects before they become problems.\n3. **Test Data Generation**: Langsmith can generate test data based on the code under test, ensuring that the tests are comprehensive and cover a wide range of scenarios.\n4. **Equivalence Partitioning**: LangSmith can help in equivalence partitioning by generating input data that covers different partitions of the input space, ensuring that the tests cover all possible scenarios.\n5. **Boundary Value Analysis**: LangSmith can assist with boundary value analysis by identifying potential issues and edge case

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'm delighted to share with you the many ways Langsmith can assist with testing.\n\n1. **Automated Documentation Generation**: Langsmith's AI-powered engine can automatically generate documentation from code comments, making it easier to create high-quality documentation while reducing manual effort. This means you can focus more on testing and less on writing lengthy documentation.\n2. **Test Case Generation**: By analyzing your code and requirements, Langsmith can help generate test cases, ensuring comprehensive coverage of your application's functionality. This saves time and reduces the risk of missing critical test scenarios.\n3. **Code Analysis for Test Coverage**: Langsmith's analysis capabilities can identify areas in your code that are not well-covered by tests. This helps you prioritize testing efforts and focus on the most important aspects of your application.\n4. **Test Data Generation**: Langsmith can assist in generating 